In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from transformers import BertTokenizer, BertModel
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score
import numpy as np

# Константы и параметры
MODEL_NAME = 'DeepPavlov/rubert-base-cased'
BATCH_SIZE = 16
MAX_LENGTH = 256
LEARNING_RATE = 2e-5

# Загрузка данных
data = pd.read_csv('processed_data.csv')
categories = [
    'Вопрос решен',
    'Нравится качество выполнения заявки',
    'Нравится качество работы сотрудников',
    'Нравится скорость отработки заявок',
    'Понравилось выполнение заявки',
    'Другое'
]
labels = data[categories].values.astype(int)

# Стратификация по первому классу для разделения данных
stratify_labels = labels.argmax(axis=1)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['comment'].values,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=stratify_labels
)

# Инициализация токенизатора
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

class CommentsDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text= str(self.texts[idx])
        label= self.labels[idx]
        encoding= tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=MAX_LENGTH,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.FloatTensor(label)
        }

# Создание датасетов и загрузчиков данных
train_dataset = CommentsDataset(train_texts, train_labels)
val_dataset = CommentsDataset(val_texts, val_labels)

# Расчет весов для балансировки классов и выборки
class_counts = train_labels.sum(axis=0)
total_samples = len(train_labels)
epsilon=1e-6

class_weights = (total_samples - class_counts + epsilon) / (class_counts + epsilon)
class_weights_tensor= torch.FloatTensor(class_weights)

sample_weights = []
for label in train_labels:
    class_indices= np.where(label==1)[0]
    weights_for_classes= class_weights[class_indices] if len(class_indices)>0 else np.array([1.0])
    sample_weight= np.min(weights_for_classes)
    sample_weights.append(sample_weight)

sampler= WeightedRandomSampler(weights=torch.FloatTensor(sample_weights), num_samples=len(sample_weights), replacement=True)

train_loader= DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler)
val_loader= DataLoader(val_dataset, batch_size=BATCH_SIZE)

# Определение модели с несколькими выходами (multi-label)
class BertMultiLabelClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained(MODEL_NAME)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, len(categories))
    
    def forward(self, input_ids, attention_mask):
        pooled_output= self.bert(input_ids=input_ids, attention_mask=attention_mask).pooler_output  
        pooled_output= self.dropout(pooled_output)
        return self.classifier(pooled_output)

device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model= BertMultiLabelClassifier().to(device)

criterion= nn.BCEWithLogitsLoss(pos_weight=class_weights_tensor.to(device))
optimizer= optim.AdamW(model.parameters(), lr=LEARNING_RATE)

# Попытка загрузить лучшую модель по F1 (если есть)
try:
    model.load_state_dict(torch.load('best_f1micro.pth'))
except FileNotFoundError:
    pass

# Функции для оценки модели
def evaluate(model, dataloader):
    model.eval()
    all_preds=[]
    all_true=[]
    with torch.no_grad():
        for batch in dataloader:
            input_ids=batch['input_ids'].to(device)
            attention_mask=batch['attention_mask'].to(device)
            labels=batch['labels'].cpu().numpy()

            outputs=model(input_ids=input_ids, attention_mask=attention_mask)
            preds=torch.sigmoid(outputs).cpu().numpy()

            all_preds.extend(preds)
            all_true.extend(labels)
    return np.array(all_true), np.array(all_preds)

print('--------------------------------------------------------------------------')

# Оценка на валидационной выборке
true_labels, preds_array = evaluate(model, val_loader)

pred_labels=(preds_array>=0.5).astype(int)

# Вычисление F1-score по классам и средних значений
f1_macro=f1_score(true_labels,pred_labels,average='macro')
f1_micro=f1_score(true_labels,pred_labels,average='micro')

for i,cate in enumerate(categories):
    try:
        score=f1_score(true_labels[:,i], pred_labels[:,i])
        print(f"F1-score для '{cate}': {score:.4f}")
    except Exception:
        print(f"Ошибка при вычислении F1 для '{cate}'.")

print(f"\nF1-score (macro): {f1_macro:.4f}")
print(f"F1-score (micro): {f1_micro:.4f}")

print('--------------------------------------------------------------------------')

# ROC-AUC по классам с обработкой ошибок
roc_auc_scores=[]
for i,cate in enumerate(categories):
    try:
        score=roc_auc_score(true_labels[:,i], preds_array[:,i])
        roc_auc_scores.append(score)
        print(f"ROC-AUC для '{cate}': {score:.4f}")
    except ValueError as e:
        print(f"ROC-AUC для '{cate}': недоступен ({e})")
if roc_auc_scores:
    print(f"\nСредний ROC-AUC по классам: {np.mean([score for score in roc_auc_scores if score is not None]):.4f}")
else:
    print("\nНет доступных значений ROC-AUC для вычисления среднего.")

d:\Py_progekt\Intensive_4\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a 

--------------------------------------------------------------------------
F1-score для 'Вопрос решен': 0.4950
F1-score для 'Нравится качество выполнения заявки': 0.5294
F1-score для 'Нравится качество работы сотрудников': 0.8931
F1-score для 'Нравится скорость отработки заявок': 0.8987
F1-score для 'Понравилось выполнение заявки': 0.4301
F1-score для 'Другое': 0.7805

F1-score (macro): 0.6711
F1-score (micro): 0.7367
--------------------------------------------------------------------------
ROC-AUC для 'Вопрос решен': 0.7590
ROC-AUC для 'Нравится качество выполнения заявки': 0.8845
ROC-AUC для 'Нравится качество работы сотрудников': 0.9663
ROC-AUC для 'Нравится скорость отработки заявок': 0.9593
ROC-AUC для 'Понравилось выполнение заявки': 0.7998
ROC-AUC для 'Другое': 0.9196

Средний ROC-AUC по классам: 0.8814
